In [21]:
import os
import csv
from pybedtools import BedTool
import numpy as np

collect all files

In [22]:
path = "/private8/Projects/itamar/ZNF/Sra_GSE73211_35_samples/REDI/nodups_cov5_HE_PLUS_STAR100len/Edits_only/"
all_files = []
for outf in [path+f for f in os.listdir(path) if f.endswith('.out.tab')]:
    with open(outf) as ffd:
        DictReader_obj = csv.DictReader(ffd, delimiter="\t")
        # content = ffd.readlines()
        file_as_list_dict = [line for line in DictReader_obj]
        all_files.append(file_as_list_dict)


count presence of every site

In [23]:
sites_counter_dict = {}
for sample in all_files:
    for site in sample:
        siteID = site["Region"]+"_"+site["Position"]
        if siteID not in sites_counter_dict.keys():
            sites_counter_dict[siteID] = 1
        else:
            sites_counter_dict[siteID] += 1

create list of recurrent sites and list of those sites for every sample

In [24]:
recurent_sites_pos = [str(key) for key in sites_counter_dict.keys() if sites_counter_dict[key] > 10]
all_files_recurent_sites = []
for sample in all_files:
    samples_recurrent_sites = [site for site in sample if (site["Region"]+"_"+site["Position"]) in recurent_sites_pos]
    all_files_recurent_sites.append(samples_recurrent_sites.copy())


filter snps

In [25]:

#filter 100% editing sites
all_files_recurent_sites_no_snp_raw =[]
for sample in all_files_recurent_sites:
        temp = [site for site in sample if 0.001 < float(site["Frequency"]) < 0.98] 
        # TODO also filte 45 -55%?
        #   = [site for site in sample if ((0.001 < float(site["Frequency"]) < 0.98) and  (0.55 < float(site["Frequency"]) or float(site["Frequency"]) < 0.45))]
        all_files_recurent_sites_no_snp_raw.append(temp.copy())

#fiter snps from dbsnp
temp_F_path = path+"temp_pos.bed"
sites_as_bed =  [site["Region"]+"\t"+str(int(site["Position"])-1)+"\t"+site["Position"]+"\n" for sample in all_files_recurent_sites_no_snp_raw for site in sample]
uniq = set(sites_as_bed)
with open(temp_F_path, "w") as outFD:
    outFD.writelines(uniq)
# intersect with snps
snps = BedTool("/private/common/Data/dbSNP-153/hg38-dbSNP153-common.bed")
sites = BedTool(temp_F_path)
result_path = path+"temp_result.bed"
# write and read results
(sites-snps).saveas(result_path)
with open(result_path, "r") as resFD:
    as_bed_no_snp = resFD.readlines()
# remove temp file
os.remove(result_path)
os.remove(temp_F_path)
# save the result as positions
no_snp_positions = [(pos.split(sep="\t")[0], pos.split(sep="\t")[2][:-1])for pos in as_bed_no_snp]
all_files_recurent_sites_no_snp = []
# save for every sample the passed sites
for sample in all_files_recurent_sites:
    all_files_recurent_sites_no_snp.append([site for site in sample if (site["Region"], site["Position"]) in no_snp_positions])
# save list of all reults (contains duplicats!)
all_sites_list_no_snp = [site for sample in all_files_recurent_sites_no_snp for site in sample]


In [26]:
all_sites_list_no_snp[1]

{'Region': 'chr12',
 'Position': '54374112',
 'Reference': 'A',
 'Strand': '0',
 'Coverage-q25': '57',
 'MeanQ': '36.88',
 'BaseCount[A,C,G,T]': '[56, 0, 1, 0]',
 'AllSubs': 'AG',
 'Frequency': '0.02'}

create table of all sites composing data for every uniq site

In [27]:
# create uniq list of all sites
uniq_sites = []
sites_dict = dict()
already = set()
for site in all_sites_list_no_snp:
    # make set from subs
    site["AllSubs"] = set(site["AllSubs"].split(" "))
    # keep only A2G sites
    if "AG" not in site["AllSubs"]:
        continue


    site["BaseCount[A,C,G,T]"]  = np.array([int(x) for x in site["BaseCount[A,C,G,T]"].replace("[","").replace("]","").replace(" ","").split(",")])
    site['Coverage-q25'] = float(site['Coverage-q25'])
    site['MeanQ'] = float(site['MeanQ'])

    siteID = site["Region"]+"_"+site["Position"] 
    if siteID not in sites_dict:
        sites_dict[siteID] = site
    else:
        sites_dict[siteID]["AllSubs"].union(site["AllSubs"])
        sites_dict[siteID]["BaseCount[A,C,G,T]"] += site["BaseCount[A,C,G,T]"]
        total_cov = sites_dict[siteID]['Coverage-q25'] + site['Coverage-q25']
        sites_dict[siteID]['MeanQ'] = round(site['MeanQ']*(site['Coverage-q25']/total_cov) + sites_dict[siteID]['MeanQ']*(sites_dict[siteID]['Coverage-q25']/total_cov),1)
        sites_dict[siteID]['Coverage-q25'] += site['Coverage-q25']
    # if (site["Region"], site["Position"]) not in already:
    #     already.add((site["Region"], site["Position"]))
    #     uniq_sites.append(site)
# update frequency 
for site in sites_dict.values():
    site["Frequency"] = round(site["BaseCount[A,C,G,T]"][2]/np.sum(site["BaseCount[A,C,G,T]"]),4)
    # convert BaseCount to string
    site["BaseCount[A,C,G,T]"] = ",".join([str(x) for x in site["BaseCount[A,C,G,T]"].tolist()])
# convert dict to list
uniq_sites = [site for site in sites_dict.values()]
print("total: ",len(all_sites_list_no_snp),"\n uniq: ", len(uniq_sites))
#save file
with open(path+"all_uniq_recurent_sites.known.tab", "w") as outFd:
    writer = csv.DictWriter(outFd, delimiter="\t",fieldnames=list(uniq_sites[0].keys()))
    writer.writeheader()
    writer.writerows(uniq_sites) 


total:  23288 
 uniq:  1072


In [28]:
uniq_sites[1]

{'Region': 'chr12',
 'Position': '124012344',
 'Reference': 'A',
 'Strand': '1',
 'Coverage-q25': 1233.0,
 'MeanQ': 43.7,
 'BaseCount[A,C,G,T]': '1214,0,19,0',
 'AllSubs': {'AG'},
 'Frequency': 0.0154}